# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re

In [250]:
# 데이터 불러오기
# 2017~2019
df_lst = []
names = ['2017','2018','201901_06','201907_12']
for name in names:
    address = "/Users/luci031/Desktop/Coding/g_auction/data_origin/auction_"+name+".csv"
    if name == '201907_12':
        a = pd.read_csv(address,encoding='euc_kr',sep=',')
    else:
        a = pd.read_csv(address,encoding='euc_kr',sep=';')
    df_lst.append(a)

# 2020~    
for year in range(2020,2023):
    if year < 2022: lim = 13 
    else: lim = 11
    for i in range(1,lim):
        if i < 13:
            if i < 10:
                df_name = "auction_"+str(year)+"0"+str(i)
            else:
                df_name = "auction_"+str(year)+str(i)
        a = pd.read_csv("/Users/luci031/Desktop/Coding/g_auction/data_origin/"+df_name+".csv",encoding='euc_kr')
        df_lst.append(a)

    # 데이터 전처리
for df in df_lst:
    df.reset_index(inplace=True,drop=True)
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'],inplace=True)

# 모든 데이터 통합
df = df_lst[0]
for dfs in df_lst[1:]:
    df = pd.concat([df,dfs])

df.reset_index(inplace=True,drop=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_6298/1118558165.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv(address,encoding='euc_kr',sep=',')


# Pre-Processing

In [3]:
df = pd.read_csv("/Users/luci031/Downloads/농수산물_경매_2018.csv",encoding="euc_kr")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3850, saw 2


In [254]:
df[(df['품목명'].str.contains('갓')) & (df['입력일']==20200213)]

,품목명,단위,등급,가격,산지,친환경구분,입력일
9232754,[갓]돌갓,8kg,특(1등),10100,경기 포천시,일반,20200213
9232755,[갓]돌갓,8kg,특(1등),9500,경기 포천시,일반,20200213
9232756,[갓]갓,10kg,특(1등),11000,전남 무안군,일반,20200213
9232757,[갓]갓,10kg,특(1등),13000,전남 무안군,일반,20200213


In [255]:
df

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[포도]샤인마스캇,4kg,특(1등),33000,충북 영동군,일반,20171101
1,[감]감(대봉),10kg,특(1등),7500,경남 진주시,일반,20171101
2,[포도]샤인마스캇,2kg,특(1등),18000,충북 영동군,일반,20171101
3,[감]감(대봉),10kg,특(1등),7500,경남 진주시,일반,20171101
4,[포도]샤인마스캇,2kg,특(1등),15000,충북 영동군,일반,20171101
...,...,...,...,...,...,...,...
29325549,[홍합국산]홍합국산,10kg,보통(3등),14750,전남 여수\t,일반,20221031
29325550,[홍합국산]홍합국산,10kg,보통(3등),15000,"경남 마산(고성,진해)",일반,20221031
29325551,[홍합국산]홍합국산,10kg,보통(3등),27000,"경남 마산(고성,진해)",일반,20221031
29325552,[홍합국산]홍합국산,10kg,보통(3등),14750,전남 여수\t,일반,20221031


In [256]:
# 결측치 확인
df.isnull().sum()

품목명           0
단위            0
등급            0
가격         4426
산지       114353
친환경구분         0
입력일           0
dtype: int64

- 결측치 처리

In [257]:
# 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [258]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [259]:
# 등급명 변환
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [260]:
df2['new_class'].value_counts()

1    22548594
2     2897390
3     1232321
9     1151580
4      127163
5      123190
6       39093
7       13723
8        3978
Name: new_class, dtype: int64

### eco

In [261]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"
df2.loc[df2["eco"]=="저농약","eco"] = "low"
df2.loc[df2["eco"]=="산지안전성","eco"] = "safe"

In [262]:
df2['eco'].value_counts()

normal         25049097
good_prd        1771161
ecofriendly     1028131
certified        199744
ing               51764
organic           28511
low                8601
safe                 23
Name: eco, dtype: int64

### reg_date

In [263]:
df2['reg_date'] = df2['reg_date'].astype('str')
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

### price

In [264]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [265]:
df2['price'] = pd.to_numeric(df2['price'])

In [266]:
df2[df2['price']<0]

,prd,scale,price,origin,eco,reg_date,new_class
9206255,[버섯]버섯(표고버섯),8.00000,-56000,경기 연천군,normal,2020-09-29,1
9223616,[버섯]버섯(표고버섯),8.00000,-56000,경기 연천군,normal,2020-09-30,1


In [267]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count   28137032.00000
mean       22981.54938
std       135193.20269
min       -56000.00000
25%         7700.00000
50%        14000.00000
75%        25000.00000
max     99999999.00000
Name: price, dtype: float64

In [268]:
df2 = df2.drop(df2[df2['price']<500].index)
df2 = df2.drop(df2[df2['scale']==0].index)

In [269]:
# kg 당 가격 생성
df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [270]:
df2['price_kg'].describe()

count   28075568.00000
mean        3870.47820
std        21322.20984
min            0.08000
25%         1400.00000
50%         2500.00000
75%         4600.00000
max     49999999.50000
Name: price_kg, dtype: float64

In [271]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[오이]백다다기           790983
[호박]애호박            493104
[깻잎]깻잎             463782
[딸기]설향             419273
[버섯]표고버섯           389057
                    ...  
[기타양채]기타양채(레드쉬)         1
[취나물]취나물(참취나물)          1
[기타엽채]열무(엽채)            1
[부추]조선부추                1
[황새치국산]황새치국산            1
Name: prd, Length: 3700, dtype: int64

In [272]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [273]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [274]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [275]:
df2 = df2[(df2['prd'].str.contains('굴')) | 
    (df2['prd'].str.contains('생강')) | 
    (df2['prd'].str.contains('마늘')) | 
    (df2['prd'].str.contains('양파')) | 
    (df2['prd'].str.contains('사과')) | 
    (df2['prd'].str.contains('배')) | 
    (df2['prd'].str.contains('배추')) | 
    (df2['prd'].str.contains('무')) | 
    (df2['prd'].str.contains('총각무')) | 
    (df2['prd'].str.contains('쪽파')) | 
    (df2['prd'].str.contains('갓')) | 
    (df2['prd'].str.contains('미나리'))]

In [276]:
df[df['prd'].str.contains('알배')]['prd'].unique()

array(['[배추]알배기배추', '[배추]알배기', '[얼갈이배추]알배기배추', '[배추]배추(알배기)',
       '[알배기]알배기(알배기)', '[기타엽채]기타엽채(알배기)'], dtype=object)

In [277]:
df2['prd'].unique()

array(['총각무', '양파', '쪽파', '무', '갓', '무우', '무청', '무화과', '배', '배추', '순무',
       '사과', '빨간양배추', '양배추', '마늘', '쑥갓', '생강', '미나리', '얼갈이배추', '열무',
       '봄동배추', '무순', '알배', '엄나무', '참죽나무순', '각굴', '굴', '봉지굴', '벅굴'],
      dtype=object)

In [278]:
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"
df2.loc[df2["prd"]=="벅굴","prd"] = "굴"

# 배추 관련 품목들 정리
df2.loc[df2["prd"]=="봄동배추","prd"] = "배추"
df2.loc[df2["prd"]=="얼갈이배추","prd"] = "배추"
df2.loc[df2['prd'].str.contains('알배'),"prd"] = "배추"

In [279]:
# 김장 품목 선택
df2 = df2[(df2['prd']=='굴') | 
    (df2['prd']=='생강') | 
    (df2['prd']=='마늘') | 
    (df2['prd']=='양파') | 
    (df2['prd']=='사과') | 
    (df2['prd']=='배') | 
    (df2['prd']=='배추') | 
    (df2['prd']=='무') | 
    (df2['prd']=='열무') | 
    (df2['prd']=='총각무') | 
    (df2['prd']=='쪽파') | 
    (df2['prd']=='갓') | 
    (df2['prd']=='미나리')]

In [280]:
df2['prd'].value_counts()

사과     729651
배추     694845
무      504028
양파     399728
배      295816
미나리    195944
쪽파     150484
마늘     125506
갓       74244
생강      56172
열무      49924
굴       45507
총각무     11405
Name: prd, dtype: int64

In [281]:
df3 = df2

### scale

In [282]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([    2.  ,    15.  ,    10.  ,     1.  ,    12.  ,     4.  ,
           8.  ,    30.  ,    20.  ,     1.5 ,     0.5 ,    25.  ,
          18.  ,  8000.  ,     5.  ,  5000.  ,  6000.  ,  3000.  ,
           6.  ,     1.3 ,     3.  ,    21.  ,     7.  ,     0.2 ,
          11.  ,    14.  ,    27.  ,    13.  ,    17.  ,     6.5 ,
          24.  ,     9.  ,     0.4 ,    16.  ,    22.  ,  1000.  ,
           8.5 ,     3.5 ,    35.  , 10000.  ,     7.5 ,     2.5 ,
           0.8 ,     0.6 ,    19.  , 10320.  ,    41.  ,    23.  ,
          31.  ,    28.  ,    99.  ,     9.5 ,    16.5 ,     4.5 ,
       11000.  ,     4.8 ,    32.  ,    26.  , 60000.  ,     0.45,
           5.7 ,    29.  ,    12.5 , 10500.  ,     0.1 ,    33.  ,
           0.9 ,     0.3 ,     1.8 ,    37.  ,     2.7 ,     3.9 ,
         820.  ,     2.2 ,     9.2 ,     4.7 ,     1.2 ,     7.8 ,
           3.2 ,     0.7 ,    74.5 ,     7.2 ,    14.1 ,     5.5 ,
           6.4 ,    34.  ,  2000.  ,    42.  ,  4000.  ,    38

In [283]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
1797,무,8000.00000,3500000,전북 고창군,normal,2017-11-01,1,437.50000
1799,무,8000.00000,2700000,충남 당진시,normal,2017-11-01,1,337.50000
5065,무,8000.00000,3000000,제주,normal,2017-11-01,1,375.00000
5067,무,5000.00000,3800000,충남 당진시,normal,2017-11-01,1,760.00000
5069,무,5000.00000,2700000,충남 당진시,normal,2017-11-01,1,540.00000
...,...,...,...,...,...,...,...,...
27750151,배,155.00000,30000,경기 안성시,normal,2022-01-07,3,193.55000
27750154,배,155.00000,25000,경기 안성시,normal,2022-01-07,3,161.29000
28082507,사과,63.00000,20000,경북 상주시,normal,2022-09-29,3,317.46000
28082508,사과,63.00000,21000,경북 상주시,normal,2022-09-29,3,333.33000


In [284]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]['prd'].unique()

array(['무', '배추', '생강', '배', '갓', '마늘', '쪽파', '양파', '사과', '열무', '미나리',
       '굴'], dtype=object)

In [285]:
# 무
display(df3[(df3['prd']=='무')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='무')]['price_kg'].describe())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='무')&(df3['price_kg']<346)].index)

count    504028.00000
mean       1187.21926
std       12738.34314
min           0.35000
25%         460.00000
50%         818.75000
75%        1390.00000
max     5555500.00000
Name: price_kg, dtype: float64

count   11541.00000
mean      607.58608
std       494.41976
min         0.35000
25%       346.00000
50%       500.00000
75%       762.00000
max     30000.00000
Name: price_kg, dtype: float64

In [286]:
# 배추
display(df3[(df3['prd']=='배추')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='배추')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='배추')]['scale'].sum())
display(df3[(df3['prd']=='배추')]['scale'].sum())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='배추')].index)

count     694845.00000
mean        1366.58065
std        26649.03051
min            1.33000
25%          625.00000
50%         1037.50000
75%         1687.50000
max     22001375.00000
Name: price_kg, dtype: float64

count    30.00000
mean    124.84300
std     191.27653
min       1.33000
25%      31.70250
50%      37.88000
75%     109.09500
max     600.00000
Name: price_kg, dtype: float64

42200.0

5219394.8999999985

In [287]:
# 생강
display(df3[(df3['prd']=='생강')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='생강')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='생강')]['scale'].sum())
display(df3[(df3['prd']=='생강')]['scale'].sum())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='생강')].index)

count    56172.00000
mean      3523.93706
std       3056.19623
min        100.00000
25%       2000.00000
50%       3200.00000
75%       4650.00000
max     500000.00000
Name: price_kg, dtype: float64

count     3.00000
mean    312.95000
std     169.70926
min     117.07000
25%     261.50500
50%     405.94000
75%     410.89000
max     415.84000
Name: price_kg, dtype: float64

1022.0

660629.3

In [288]:
# 배
display(df3[(df3['prd']=='배')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='배')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='배')]['scale'].sum())
display(df3[(df3['prd']=='배')]['scale'].sum())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='배')].index)

count   295816.00000
mean      2370.51845
std       1633.54394
min         66.67000
25%       1466.67000
50%       2133.33000
75%       3066.67000
max     400000.00000
Name: price_kg, dtype: float64

count    14.00000
mean    270.71643
std      84.00039
min     161.29000
25%     203.98500
50%     248.00000
75%     342.00000
max     416.00000
Name: price_kg, dtype: float64

1717.1

4059159.2799999984

In [289]:
# 갓
display(df3[(df3['prd']=='갓')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='갓')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='갓')]['scale'].sum())
display(df3[(df3['prd']=='갓')]['scale'].sum())

count     74244.00000
mean       1654.29313
std        8299.33113
min          25.00000
25%         800.00000
50%        1200.00000
75%        1980.00000
max     2158468.75000
Name: price_kg, dtype: float64

count      1.00000
mean    1200.00000
std            NaN
min     1200.00000
25%     1200.00000
50%     1200.00000
75%     1200.00000
max     1200.00000
Name: price_kg, dtype: float64

50.0

986918.6

In [290]:
# 마늘
display(df3[(df3['prd']=='마늘')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='마늘')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='마늘')]['scale'].sum())
display(df3[(df3['prd']=='마늘')]['scale'].sum())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='마늘')].index)

count    125506.00000
mean       4666.49539
std        6502.31461
min          50.00000
25%        2533.33000
50%        4000.00000
75%        6713.50000
max     2002800.00000
Name: price_kg, dtype: float64

count     2.00000
mean    118.00000
std      31.11270
min      96.00000
25%     107.00000
50%     118.00000
75%     129.00000
max     140.00000
Name: price_kg, dtype: float64

100.0

1621715.36

In [291]:
# 쪽파
display(df3[(df3['prd']=='쪽파')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='쪽파')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='쪽파')]['scale'].sum())
display(df3[(df3['prd']=='쪽파')]['scale'].sum())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='쪽파')].index)

count    150484.00000
mean       4462.82200
std        8167.04007
min          60.00000
25%        2500.00000
50%        3700.00000
75%        5500.00000
max     2300000.00000
Name: price_kg, dtype: float64

count     3.00000
mean    259.07333
std      32.96448
min     237.62000
25%     240.09500
50%     242.57000
75%     269.80000
max     297.03000
Name: price_kg, dtype: float64

303.0

1402985.1599999997

In [292]:
# 양파
display(df3[(df3['prd']=='양파')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='양파')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='양파')]['scale'].sum())
display(df3[(df3['prd']=='양파')]['scale'].sum())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='양파')&(df3['price_kg']<700)].index)

count    399728.00000
mean        839.35066
std        8752.96356
min          25.00000
25%         480.00000
50%         720.00000
75%        1058.33000
max     4216786.11000
Name: price_kg, dtype: float64

count     8.00000
mean    231.33375
std     351.49011
min      27.20000
25%      33.20000
50%      35.60000
75%     265.00250
max     800.00000
Name: price_kg, dtype: float64

2700.0

5551775.9

In [293]:
# 사과
display(df3[(df3['prd']=='사과')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='사과')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='사과')]['scale'].sum())
display(df3[(df3['prd']=='사과')]['scale'].sum())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='사과')].index)

count   729651.00000
mean      2612.13274
std       2968.56554
min         35.71000
25%       1500.00000
50%       2300.00000
75%       3300.00000
max     999999.90000
Name: price_kg, dtype: float64

count    23.00000
mean    301.77739
std     171.04528
min      35.71000
25%     195.45500
50%     285.71000
75%     390.91000
max     781.82000
Name: price_kg, dtype: float64

2163.0

7088901.419999999

In [294]:
# 열무
display(df3[(df3['prd']=='열무')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='열무')]['price_kg'].describe())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='열무')].index)

count     49924.00000
mean       1037.03078
std        9591.13641
min          40.00000
25%         600.00000
50%         850.00000
75%        1250.00000
max     2125300.00000
Name: price_kg, dtype: float64

count    1.00000
mean    60.00000
std          NaN
min     60.00000
25%     60.00000
50%     60.00000
75%     60.00000
max     60.00000
Name: price_kg, dtype: float64

In [295]:
# 미나리
display(df3[(df3['prd']=='미나리')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='미나리')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='미나리')]['scale'].sum())
display(df3[(df3['prd']=='미나리')]['scale'].sum())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='미나리')].index)

count    195944.00000
mean       3868.71495
std        4472.85467
min          14.29000
25%        1866.67000
50%        3225.00000
75%        5200.00000
max     1200000.00000
Name: price_kg, dtype: float64

count     8.00000
mean    400.83375
std     177.13600
min     120.00000
25%     302.50000
50%     460.00000
75%     517.50000
max     586.67000
Name: price_kg, dtype: float64

725.0

1487927.4

In [296]:
# 굴
display(df3[(df3['prd']=='굴')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='굴')]['price_kg'].describe())
display(df3[(df3['scale']>=50) & (df3['prd']=='굴')]['scale'].sum())
display(df3[(df3['prd']=='굴')]['scale'].sum())
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='굴')].index)

count   45507.00000
mean     8326.07913
std      3669.92385
min       100.00000
25%      6400.00000
50%      8000.00000
75%     10000.00000
max     67650.00000
Name: price_kg, dtype: float64

count     2.00000
mean    310.00000
std      14.14214
min     300.00000
25%     305.00000
50%     310.00000
75%     315.00000
max     320.00000
Name: price_kg, dtype: float64

100.0

164583.4

### origin

In [297]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['경기 평택시', '경기 화성시', '충남 태안군', '충남 천안시', '충남 서산시', '전북 고창군',
       '전남 영암군', '중국', '수입산 중국', '중국(CN)', '제주도 제주시', '충남 예산군', '경기도 김포시',
       '경기도 남양주시', '충남 홍성군', '경기 남양주', '충남 서천군', '충남 아산시', '경기 포천시',
       '경기 구리시', '경기 동두천시', '경기 성남시', '경기 고양시', '경기 고양시 일산서구', '경기 남양주시',
       '경기 안성시', '경기도 고양시', '서울시 은평구', '전남 여수시', '서울 은평구', '서울', '경북 영주시',
       '전남 무안군', '강원 평창군', '강원 홍천군', '충남 당진시', '경기 가평군', '경기 파주시', '세종',
       '경북 상주시', '세종 세종시', '전남 보성군', '강원 횡성군', '경기 연천군', '충남 보령시',
       '전북 완주군', '전남 함평군', '전남 신안군', '전북 장수군', '전북 무주군', '경북 청송군',
       '경북 의성군', '충북 충주시', '충북 단양군', '경북 김천시', '전남 나주시', '서울 중랑구',
       '경기 여주군', '경기 여주시', '전북 부안군', '제주', '강원 영월군', '강원 춘천시', '경기 이천시',
       '충남 청양군', '경북 영천시', '충북 영동군', '경북 안동시', '대구 북구', '서울 송파구',
       '경북 봉화군', '경북 청도군', '충북 괴산군', '충북 제천시', '경북 문경시', '경남 거창군',
       '서울 도봉구', '경기 양평군', '강원 철원군', '충남 연기군', '전북 전주시', '경기 양주시',
       '경기 하남시', '강원 원주시', '경기 광명시', '경기 광주시', '경기 용인시', '경남 창녕군',
       '전남 해남군', '제주 제주시',

In [298]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2
280,총각무,2.00000,1700,경기 평택시,normal,2017-11-01,1,850.00000,경기,평택시,NaN
282,총각무,2.00000,1500,경기 평택시,normal,2017-11-01,1,750.00000,경기,평택시,NaN
284,총각무,2.00000,1800,경기 평택시,normal,2017-11-01,1,900.00000,경기,평택시,NaN
286,총각무,2.00000,1200,경기 평택시,normal,2017-11-01,1,600.00000,경기,평택시,NaN
288,총각무,2.00000,1100,경기 화성시,normal,2017-11-01,1,550.00000,경기,화성시,NaN
...,...,...,...,...,...,...,...,...,...,...,...
28137016,양파,15.00000,11000,전남 무안군,normal,2022-10-27,8,733.33000,전남,무안군,NaN
28137017,양파,15.00000,11000,전남 무안군,normal,2022-10-27,8,733.33000,전남,무안군,NaN
28137020,양파,15.00000,17000,전남 무안군,normal,2022-10-28,8,1133.33000,전남,무안군,NaN
28137021,양파,15.00000,17000,전남 무안군,normal,2022-10-28,8,1133.33000,전남,무안군,NaN


In [299]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=2)

In [300]:
df4.head(1)

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city
280,총각무,2.00000,1700,경기 평택시,normal,2017-11-01,1,850.00000,경기,평택시


In [301]:
df4['state'].unique()

array(['경기', '충남', '전북', '전남', '중국', '수입산', '중국(CN)', '제주도', '경기도', '서울시',
       '서울', '경북', '강원', '세종', '충북', '제주', '대구', '경남', '기타외국', '광주',
       '강원도', '인천', '부산', '인천시', '울산', '없음', '해당사항없', '광주시', '충북청주시상당구',
       '대전', '전라북도', '충청남도', '충북청주시흥덕구', '서울특별시', '가락동', '대구시',
       '경기고양시일산구', '세종시', '국산', '공통출하처', '경상북도', '부산시', '대전시', '경기성남시수정구',
       '전라남도', '부천시', '성남시', '전주시', '뉴질랜드', '경북/문경시', '세종특별자치시', '미국',
       '경상남도', '충청북도', '베트남', '대만', '태국', '호주', '제주/서귀포', '청주시', '경북/안동시',
       '전남/무안군', '경북/구미시', '경북/상주시', '포항시', '경북/영천시', '경남/합천군', '제주시',
       '수원시', '서귀포시', '충남/아산시', '경남/남해군', '광주광역시', '인천광역시', '충남/홍성군',
       '전남/고흥군', '일본', '제주특별자치', '이시원', '부산광역시', '국내산', '수입', '국내산\t'],
      dtype=object)

In [302]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [303]:
df4["state"].unique()

array(['경기', '충남', '전북', '전남', '중국', '수입산', '중국(CN)', '제주', '서울', '경북',
       '강원', '세종', '충북', '대구', '경남', '기타외국', '광주', '인천', '부산', '울산', '없음',
       '해당사항없', '충북청주', '대전', '전라북', '충청남', '서울특별', '가락동', '경기고양', '국산',
       '공통출하처', '경상북', '경기성남', '전라남', '부천', '성남', '전주', '뉴질랜드', '경북/문경',
       '세종특별자치', '미국', '경상남', '충청북', '베트남', '대만', '태국', '호주', '제주/서귀포',
       '청주', '경북/안동', '전남/무안군', '경북/구미', '경북/상주', '포항', '경북/영천', '경남/합천군',
       '수원', '서귀포', '충남/아산', '경남/남해군', '광주광역', '인천광역', '충남/홍성군', '전남/고흥군',
       '일본', '제주특별자치', '이', '부산광역', '국내산', '수입'], dtype=object)

In [304]:
# state
# 특이케이스 제거
df4 = df4.drop(df4[(df4['state']=='없음')].index)
df4 = df4.drop(df4[(df4['state']=='해당사항없')].index)

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"
df4.loc[df4["state"]=="공통출하처","city"] = "미상"
df4.loc[df4["state"]=="공통출하처","state"] = "국산"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="부산광역","city"] = "부산"
df4.loc[df4["state"]=="부산광역","state"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="세종특별자치","city"] = "세종"
df4.loc[df4["state"]=="세종특별자치","state"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","city"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","state"] = "제주"
df4.loc[df4["state"]=="서귀포","city"] = "제주"
df4.loc[df4["state"]=="서귀포","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"
df4.loc[df4["state"]=="광주광역","state"] = "광주"
df4.loc[df4["state"]=="광주광역","city"] = "광주"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="경기성남","city"] = "성남"
df4.loc[df4["state"]=="경기성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="충북청주시흥덕구","city"] = "청주"
df4.loc[df4["state"]=="충북청주시흥덕구","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"
df4.loc[df4["state"]=="경북/안동","city"] = "안동"
df4.loc[df4["state"]=="경북/안동","state"] = "경북"
df4.loc[df4["state"]=="전남/무안군","state"] = "전남"
df4.loc[df4["state"]=="전남/무안군","city"] = "무안"
df4.loc[df4["state"]=="무안","state"] = "전남"
df4.loc[df4["state"]=="무안","city"] = "무안"
df4.loc[df4["state"]=="경북/구미","city"] = "구미"
df4.loc[df4["state"]=="경북/구미","state"] = "경북"
df4.loc[df4["state"]=="경북/상주","city"] = "상주"
df4.loc[df4["state"]=="경북/상주","state"] = "경북"
df4.loc[df4["state"]=="경북/영천","city"] = "영천"
df4.loc[df4["state"]=="경북/영천","state"] = "경북"
df4.loc[df4["state"]=="경남/합천군","city"] = "합천"
df4.loc[df4["state"]=="경남/합천군","state"] = "경남"
df4.loc[df4["state"]=="전남/고흥군","city"] = "고흥"
df4.loc[df4["state"]=="전남/고흥군","state"] = "전남"
df4.loc[df4["state"]=="경남/남해군","state"] = "경남"
df4.loc[df4["state"]=="경남/남해군","city"] = "남해"
df4.loc[df4["state"]=="충남/아산","state"] = "충남"
df4.loc[df4["state"]=="충남/아산","city"] = "아산"
df4.loc[df4["state"]=="충남/홍성군","state"] = "충남"
df4.loc[df4["state"]=="충남/홍성군","city"] = "아산"
df4.loc[df4["state"]=="서해안","city"] = "미상"
df4.loc[df4["state"]=="서해안","state"] = "국산"
df4.loc[df4["state"]=="남해안","city"] = "미상"
df4.loc[df4["state"]=="남해안","state"] = "국산"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="중국(CN)","city"] = "중국"
df4.loc[df4["state"]=="중국(CN)","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[df4["state"]=="캄보디아","city"] = "캄보디아"
df4.loc[df4["state"]=="캄보디아","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"
df4.loc[df4["state"]=="브라질","city"] = "브라질"
df4.loc[df4["state"]=="브라질","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"
df4.loc[df4["state"]=="이탈리아","city"] = "이탈리아"
df4.loc[df4["state"]=="이탈리아","state"] = "수입"
df4.loc[df4["state"]=="불가리아","city"] = "불가리아"
df4.loc[df4["state"]=="불가리아","state"] = "수입"
df4.loc[df4["state"]=="그리스","city"] = "그리스"
df4.loc[df4["state"]=="그리스","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"
df4.drop(df4[df4["state"]=="미등록"].index,inplace=True)

In [305]:
df4[(df4['prd'].str.contains('갓')) & (df4['reg_date']==dt.datetime(2020,2,13))]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city
7360116,갓,8.00000,10100,경기 포천시,normal,2020-02-13,1,1262.50000,경기,포천시
7360117,갓,8.00000,9500,경기 포천시,normal,2020-02-13,1,1187.50000,경기,포천시
7360118,갓,10.00000,11000,전남 무안군,normal,2020-02-13,1,1100.00000,전남,무안군
7360119,갓,10.00000,13000,전남 무안군,normal,2020-02-13,1,1300.00000,전남,무안군


In [306]:
df4 = df4.drop(df4[(df4['origin']=='이시원')].index)

In [307]:
df4['state'].unique()

array(['경기', '충남', '전북', '전남', '수입', '제주', '서울', '경북', '강원', '세종', '충북',
       '대구', '경남', '광주', '인천', '부산', '울산', '대전', '국산'], dtype=object)

In [308]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [309]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [310]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [311]:
df4['city'].unique()

array(['평택', '화성', '태안', '천안', '서산', '고창', '영암', '중국', '제주', '예산', '김포',
       '남양주', '홍성', '서천', '아산', '포천', '구리', '동두천', '성남', '고양', '안성',
       '은평구', '여수', '미상', '영주', '무안', '평창', '홍천', '당진', '가평', '파주', '세종',
       '상주', '보성', '횡성', '연천', '보령', '완주', '함평', '신안', '장수', '무주', '청송',
       '의성', '충주', '단양', '김천', '나주', '중랑구', '여주', '부안', '영월', '춘천', '이천',
       '청양', '영천', '영동', '안동', '대구', '송파구', '봉화', '청도', '괴산', '제천', '문경',
       '거창', '도봉구', '양평', '철원', '연기', '전주', '양주', '하남', '원주', '광명', '광주',
       '용인', '창녕', '해남', '양구', '금산', '논산', '공주', '강동구', '음성', '인제', '부여',
       '영양', '밀양', '함양', '합천', '임실', '화천', '진안', '강릉', '정선', '완도', '김제',
       '양산', '서구', '부산', '광산구', '의령', '흥', '안산', '부천', '위', '산청', '남원',
       '계양구', '영덕', '장성', '고흥', '남해', '삼척', '영광', '남구', '진도', '오산', '곡성',
       '울산', '익산', '예천', '강서구', '정읍', '청주', '청원', '강화', '대전', '김', '진주',
       '장흥', '전', '구미', '태백', '아', '수원', '경주', '강진', '안양', '보은', '목포',
       '양천구', '광양', '진천', '김해', '동구', '고성', '경산', '순

In [312]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="포","city"] = "군포"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[df4["city"]=="보성(벌교)","city"] = "보성"
df4.loc[df4["city"]=="울진(후포)","city"] = "울진"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="순","city"] = "순창"
df4.loc[df4["city"]=="익","city"] = "익산"
df4.loc[df4["city"]=="미금","city"] = "성남"
df4.loc[df4["city"]=="주문진","city"] = "강릉"
df4.loc[df4["city"]=="울릉도","city"] = "울릉"
df4.loc[df4["city"]=="거제(장승포)","city"] = "거제"
df4.loc[df4["city"]=="남양","city"] = "남양주"
df4.loc[(df4["city"]=="완")&(df4["state"]=="전북"),"city"] = "완주"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
df4.loc[(df4['prd']=="호박") & (df4['state']=='전북'),"city"] = "전주"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="신"),"city"] = "신안"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="장"),"city"] = "장수"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="진"),"city"] = "진도"
df4.loc[(df4["state"]=="충남")&(df4["city"]=="부"),"city"] = "부여"
df4.loc[(df4["state"]=="경남")&(df4["city"]=="의"),"city"] = "의령"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="함"),"city"] = "함평"
df4.loc[(df4["state"]=="경북")&(df4["city"]=="영"),"city"] = "영주"
df4.loc[(df4["state"]=="경북")&(df4["city"]=="안"),"city"] = "안동"
df4.loc[(df4["state"]=="경남")&(df4["city"]=="거"),"city"] = "거창"
df4.loc[(df4["state"]=="경남")&(df4["city"]=="합"),"city"] = "합천"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="김"),"city"] = "김제"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="전"),"city"] = "전주"
df4.loc[(df4["state"]=="충남")&(df4["city"]=="아"),"city"] = "아산"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="여"),"city"] = "여수"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="고"),"city"] = "고창"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="무"),"city"] = "무안"
# 전라남도 장 -> 장흥, 장성 두개 있으므로 구분 불가, 삭제
df4.drop(df4[(df4["city"]=="장")&(df4["state"]=="전남")].index,inplace=True)

## 광역시
df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"
df4.loc[df4["city"]=="동작구","city"] = "서울"
df4.loc[df4["city"]=="용산구","city"] = "서울"
df4.loc[df4["city"]=="광진구","city"] = "서울"
df4.loc[df4["city"]=="종로구","city"] = "서울"
df4.loc[df4["city"]=="서대문구","city"] = "서울"
df4.loc[df4["city"]=="서대문","city"] = "서울"
df4.loc[df4["city"]=="성북구","city"] = "서울"
df4.loc[df4["city"]=="관악구","city"] = "서울"
df4.loc[df4["city"]=="강북구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"
df4.loc[df4["city"]=="고성(거진,대진)","city"] = "고성"
df4.loc[df4["city"]=="연기","state"] = "세종"
df4.loc[df4["city"]=="연기","city"] = "세종"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
df4.loc[df4["city"]=="사우디","city"] = "사우디아라비아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [313]:
#df2[(df2['prd']=='양파') & (df2['price']==23000) & (df2['scale']==15)]['origin'].unique()

In [314]:
df4[df4['city']=='무']['origin'].unique()

array([], dtype=object)

In [315]:
df4['city'].unique()

array(['평택', '화성', '태안', '천안', '서산', '고창', '영암', '중국', '제주', '예산', '김포',
       '남양주', '홍성', '서천', '아산', '포천', '구리', '동두천', '성남', '고양', '안성', '서울',
       '여수', '미상', '영주', '무안', '평창', '홍천', '당진', '가평', '파주', '세종', '상주',
       '보성', '횡성', '연천', '보령', '완주', '함평', '신안', '장수', '무주', '청송', '의성',
       '충주', '단양', '김천', '나주', '여주', '부안', '영월', '춘천', '이천', '청양', '영천',
       '영동', '안동', '대구', '봉화', '청도', '괴산', '제천', '문경', '거창', '양평', '철원',
       '전주', '양주', '하남', '원주', '광명', '광주', '용인', '창녕', '해남', '양구', '금산',
       '논산', '공주', '음성', '인제', '부여', '영양', '밀양', '함양', '합천', '임실', '화천',
       '진안', '강릉', '정선', '완도', '김제', '양산', '인천', '부산', '의령', '시흥', '안산',
       '부천', '군위', '산청', '남원', '영덕', '장성', '고흥', '남해', '삼척', '영광', '진도',
       '오산', '곡성', '울산', '익산', '예천', '정읍', '청주', '청원', '강화', '대전', '진주',
       '장흥', '구미', '태백', '수원', '경주', '강진', '안양', '보은', '목포', '광양', '진천',
       '김해', '고성', '경산', '순창', '순천', '함안', '군포', '의정부', '고령', '옥천', '울진',
       '성주', '칠곡', '포항', '계룡', '군산', '구례', '하동',

In [316]:
# 기존 city 리스트와 비교
ex_city = pd.read_csv("../cities.csv")
df4[~df4['city'].isin(ex_city['city'])]['city'].unique()

array([], dtype=object)

In [317]:
# 리스트 최신화
new_city = pd.DataFrame(df4['city'].unique(),columns=['city'])
cities = ex_city.append(new_city,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_6298/1856950528.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = ex_city.append(new_city,ignore_index=True)


In [318]:
cities.drop(columns=['Unnamed: 0'],inplace=True)
cities.drop_duplicates(inplace=True)
cities.reset_index(inplace=True,drop=True)

In [319]:
cities.to_csv('../cities.csv')

In [320]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [321]:
# parquet 파일로 변환하여 용량 줄이기
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3329675 entries, 280 to 28137022
Data columns (total 9 columns):
 #   Column     Dtype         
---  ------     -----         
 0   prd        object        
 1   scale      float64       
 2   price      int64         
 3   eco        object        
 4   reg_date   datetime64[ns]
 5   new_class  int64         
 6   price_kg   float64       
 7   state      object        
 8   city       object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 254.0+ MB


In [323]:
df5[(df5['prd'].str.contains('갓')) & (df5['reg_date']==dt.datetime(2020,2,13))]

,prd,scale,price,eco,reg_date,new_class,price_kg,state,city
7360116,갓,8.00000,10100,normal,2020-02-13,1,1262.50000,경기,포천
7360117,갓,8.00000,9500,normal,2020-02-13,1,1187.50000,경기,포천
7360118,갓,10.00000,11000,normal,2020-02-13,1,1100.00000,전남,무안
7360119,갓,10.00000,13000,normal,2020-02-13,1,1300.00000,전남,무안


In [322]:
address2 = "/Users/luci031/Desktop/Coding/g_auction/data_ingredients/auction_total.parquet"
df5.to_parquet(address2,engine="pyarrow", compression='gzip')
df5.to_csv("/Users/luci031/Desktop/Coding/g_auction/data_proceed/auction_total.csv")